# Developing AI System

### Chatbot

A **chatbot** is a computer program that simulates human conversation, usually through text or voice. With **OpenAI**, a chatbot uses powerful language models like **GPT-4** to understand questions and generate smart, human-like answers.

In simple terms:

> A chatbot with OpenAI can chat, answer questions, summarize info, or help with tasks — almost like talking to a helpful assistant.

It works by sending messages to the model (input) and receiving a response (output), often using the `chat.completions.create()` function from the OpenAI API.



In [98]:
#This version of chatbot can only be running through the console,
#Check code app.py a version of this chatbot deployed on Streamlit Platform for a better experience
#To run the chatbot --> Terminal : streamlit run app.py

import os 
import openai
from time import sleep
from tqdm import tgrange, tqdm_notebook

#Bloc
MESSAGES = [{"role":"system", "content": "You're a Chatbot, your name is AlexBot you've been created by Alexandre Ohayon, you can answer any customer's questions by summarizing your answer."}]
#Dialog Box ask your question to the ChatBot:

try:
    print("AlexBot is ready. Tape 'exit' to quit the conversation")

    #Start a session:
    while True:
        #Enter your question:
        USER_PROMPT = input("Enter your message here: ") #Use the dialog box
        #Tape exist in lower character to quit the conversation with the Bot.
        if USER_PROMPT.lower() == "exit": 
            print("Thanks for choosing AlexBot. See you soon !") #Message from system
            break #Session closed
            
        #If a conversation exists then add user question to the Bloc in line 7:
        MESSAGES.append({"role":"user", "content": USER_PROMPT}) #Add the question into the list:
        user_question = {"role":"user", "content": USER_PROMPT}
        #print(f"User_question : {USER_PROMPT}")

                
        #Pass MESSAGES through the LLM to get response from the AI Agent:
        client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=API_KEY)
            
        #Response from the Intelligent Agent:
        # Add the appropriate parameters to the decorator
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=MESSAGES,
            max_tokens=300,
            temperature=0,
            top_p=0.9,
            stop=["\n"]
        )

        assistant_dict = {"role":"assistant", "content": response.choices[0].message.content}
        print(f"ChniniBot : {assistant_dict["content"]}")
        print("--------------------------------------------------")
        #Add the question into the list:
        MESSAGES.append(assistant_dict)

except Exception as e:
    print("Error :", e)

AlexBot is ready. Tape 'exit' to quit the conversation
ChniniBot : Salut ! Comment puis-je vous aider aujourd'hui ?
--------------------------------------------------
Thanks for choosing AlexBot. See you soon !


### Get response in Json Format 

In [104]:
import openai
import os

# Initialisation du client OpenAI avec la clé API (via variable d'environnement)
client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=API_KEY) # Remplace si besoin

def get_famous_dishes_by_country():
    # Prompt demandé
    prompt = (
        "Donne-moi une liste des 10 plats les plus connus dans le monde, "
        "triée par pays, au format JSON. Pour chaque plat, indique le nom du plat et le pays d'origine. "
        "Utilise ce format : "
        '{ "plats": [ { "nom": "Sushi", "pays": "Japon" }, ... ] }'
    )

    # Appel à l'API avec la méthode chat.completions.create
    response = client.chat.completions.create(
        model="gpt-4o",  # ou "gpt-4"
        messages=[
            {"role": "system", "content": "Tu es un assistant culinaire qui fournit des réponses en JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=500,
        response_format={"type":"json_object"} # Nécessite GPT-4-turbo ou GPT-4o
    )

    return response.choices[0].message.content

# Utilisation
if __name__ == "__main__":
    json_result = get_famous_dishes_by_country()
    print(json_result)


{
  "plats": [
    { "nom": "Sushi", "pays": "Japon" },
    { "nom": "Pizza", "pays": "Italie" },
    { "nom": "Tacos", "pays": "Mexique" },
    { "nom": "Croissant", "pays": "France" },
    { "nom": "Paella", "pays": "Espagne" },
    { "nom": "Curry", "pays": "Inde" },
    { "nom": "Poutine", "pays": "Canada" },
    { "nom": "Hamburger", "pays": "États-Unis" },
    { "nom": "Falafel", "pays": "Moyen-Orient" },
    { "nom": "Kimchi", "pays": "Corée du Sud" }
  ]
}


### Function tools 

Function tools (also called tools or function calling in OpenAI) let you connect a chatbot to external functions in your code.

🔧 In simple terms:
Function tools allow the chatbot to call real functions — like searching a database, getting the weather, or running a calculation — based on the user's question.

📌 Example:
If the user says:

"What's the weather in Paris?"

The model can call your get_weather(city) function, get the result, and reply with it.

In [137]:
print(response.choices[0].message.tool_calls[0])


TypeError: 'NoneType' object is not subscriptable

In [138]:
import os
import openai
import json

system = "You're an AI agent capable of analyzing job offers"
prompt = "Create a simple job offer for a data scientist position in Microsoft in Montreal and salary equal to 120,000$ "

try:
    # Création de l'offre
    client = openai.OpenAI(api_key=os.getenv("API_KEY"))

    response1 = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]
    )

    job_offer = response1.choices[0].message.content

    # Extraction des informations
    response2 = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": job_offer}
        ],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "extract_job_info",
                    "description": "Extract key information from a job offer: position, city, salary, and company name.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "job_position": {
                                "type": "string",
                                "description": "The job title (e.g., Data Scientist)."
                            },
                            "city": {
                                "type": "string",
                                "description": "City where the job is located."
                            },
                            "salary": {
                                "type": "string",
                                "description": "Salary for the job."
                            },
                            "company_name": {
                                "type": "string",
                                "description": "Company offering the position."
                            }
                        },
                        "required": ["job_position", "city", "salary", "company_name"]
                    }
                }
            }
        ],
        tool_choice={"type": "function", "function": {"name": "extract_job_info"}}
    )

    print("📄 Job Offer:\n", job_offer)
    print("\n------------------------------")

    # Extraire les arguments du tool_call
    tool_call = response2.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    print("🧠 Extracted Info:")
    print(f"📌 Position      : {args['job_position']}")
    print(f"🏙️  City          : {args['city']}")
    print(f"💼 Company Name : {args['company_name']}")
    print(f"💰 Salary        : {args['salary']}")

except Exception as e:
    print("❌ Error:", e)


📄 Job Offer:
 **Job Title: Data Scientist**

**Location:** Montreal, QC, Canada  
**Company:** Microsoft  
**Salary:** CAD 120,000 per year

**Job Description:**

Microsoft is seeking a skilled and enthusiastic Data Scientist to join our dynamic team in Montreal. As a Data Scientist, you will play a pivotal role in leveraging data-driven insights to drive strategic decision-making and contribute to the development of innovative solutions across various projects.

**Key Responsibilities:**

- Analyze large and complex data sets to identify trends, patterns, and actionable insights.
- Develop predictive models and algorithms to solve business challenges.
- Collaborate with cross-functional teams to define data requirements and drive data collection efforts.
- Communicate findings and recommendations to stakeholders through clear and compelling data visualizations.
- Stay updated with the latest industry trends and technologies in data science and machine learning to ensure continuous imp